# DehazeNet

In [5]:
#/content

In [3]:
!wget http://www.vision.ee.ethz.ch/ntire18/o-haze/O-HAZE.zip

--2021-02-11 17:15:58--  http://www.vision.ee.ethz.ch/ntire18/o-haze/O-HAZE.zip
Resolving www.vision.ee.ethz.ch (www.vision.ee.ethz.ch)... 129.132.19.216, 2001:67c:10ec:254::216
Connecting to www.vision.ee.ethz.ch (www.vision.ee.ethz.ch)|129.132.19.216|:80... connected.
HTTP request sent, awaiting response... 301 
Location: https://data.vision.ee.ethz.ch/cvl/ntire18//o-haze/O-HAZE.zip [following]
--2021-02-11 17:15:59--  https://data.vision.ee.ethz.ch/cvl/ntire18//o-haze/O-HAZE.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573320339 (547M) [application/zip]
Saving to: ‘O-HAZE.zip’

O-HAZE.zip          100%[===================>] 546.76M  26.5MB/s    in 21s     

2021-02-11 17:16:21 (25.5 MB/s) - ‘O-HAZE.zip’ saved [573320339/573320339]



In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import random
from PIL import Image
import time

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import Adam
BATCH_SIZE = 128
EPOCH = 10


In [21]:

class DehazeNet(Model):
  def __init__(self, input=16, groups=4):
    super(DehazeNet, self).__init__()
    self.input = input
    self.groups = groups
    self.conv1=tf.keras.layers.Conv2D(self.input,kernel_size=5)
    self.conv2=tf.keras.layers.Conv2D(16,kernel_size=3,padding=1)
    self.conv3=tf.keras.layers.Conv2D(16,kernel_size=5,padding=2)
    self.conv4=tf.keras.layers.Conv2D(16,kernel_size=7,padding=3)
    self.maxpool=tf.keras.layers.MaxPool2D(7,1)
    self.conv5=tf.keras.layers.Conv2D(1,kernel_size=6)
    self.brelu=tf.keras.activations.relu(max_value=1)

  def Maxout(self,x,groups):
    x = x.reshape(x.shape[0], groups, x.shape[1]//groups, x.shape[2], x.shape[3])
    x, y = np.max(x, dim=2, keepdim=True)
    out = x.reshape(x.shape[0],-1, x.shape[3], x.shape[4])
    return out

  def call(self, x):
    out = self.conv1(x)
    out = self.Maxout(out, self.groups)
    out1 = self.conv2(out)
    out2 = self.conv3(out)
    out3 = self.conv4(out)
    y = tf.concat((out1,out2,out3), dim=1)
		#print(y.shape[0],y.shape[1],y.shape[2],y.shape[3],)
    y = self.maxpool(y)
		#print(y.shape[0],y.shape[1],y.shape[2],y.shape[3],)
    y = self.conv5(y)
		# y = self.relu(y)
		# y = self.BRelu(y)
		#y = torch.min(y, torch.ones(y.shape[0],y.shape[1],y.shape[2],y.shape[3]))
    y = self.brelu(y)
    y = y.reshape(y.shape[0],-1)
    return y

In [23]:
'''class DehazeNet(Model):
  def __init__(self):
    super(DehazeNet, self).__init__()
    inputs = tf.keras.Input(shape = [550,413,3])
    conv1 = Conv2D(input_shape = (550, 413, 3), filters = 3, kernel_size = 1, strides = 1, padding = 'same', use_bias = True,
                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(inputs)
    conv2 = Conv2D(filters = 3, kernel_size = 3, strides = 1, padding = 'same', use_bias = True,
                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(conv1)
    concat1 = tf.concat([conv1,conv2], axis = -1)
    
    conv3 = Conv2D(filters = 3, kernel_size = 5, strides = 1, padding = 'same', use_bias = True,
                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(concat1)
    concat2 = tf.concat([conv2,conv3], axis = -1)
    
    conv4 = Conv2D(filters = 3, kernel_size = 7, strides = 1, padding = 'same', use_bias = True,
                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(concat2)
    concat3 = tf.concat([conv1,conv2,conv3,conv4], axis = -1)
    
    conv5 = Conv2D(filters = 3, kernel_size = 3, strides = 1, padding = 'same', use_bias = True,
                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(concat3)
    K = conv5
    output = ReLU(max_value = 1.0)(tf.math.multiply(K,inputs) - K + 1.0)
'''


"class DehazeNet(Model):\n  def __init__(self):\n    super(DehazeNet, self).__init__()\n    inputs = tf.keras.Input(shape = [550,413,3])\n    conv1 = Conv2D(input_shape = (550, 413, 3), filters = 3, kernel_size = 1, strides = 1, padding = 'same', use_bias = True,\n                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(inputs)\n    conv2 = Conv2D(filters = 3, kernel_size = 3, strides = 1, padding = 'same', use_bias = True,\n                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(conv1)\n    concat1 = tf.concat([conv1,conv2], axis = -1)\n    \n    conv3 = Conv2D(filters = 3, kernel_size = 5, strides = 1, padding = 'same', use_bias = True,\n                   kernel_initializer = tf.random_normal_initializer(stddev = 0.02), kernel_regularizer = tf.keras.regularizers.l2(1e-2))(concat1)\n    concat2 = tf.concat([conv2,conv3], ax